In [2]:
!pip install paddlepaddle-gpu paddleocr


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 1.5 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.9/758.9 MB 1.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.5/407.5 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 45.7 MB/s eta 0:00:0000:0100:01
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117029 sha256=99ace51a063f7e6715f7d25bd559dba0be317b209cef46c420451d8b00974fb7
  Stored in directory: /root/.cache/pip/wheels/d6/6d/5d/5b73fa0f46d01a793713f8859201361e9e581ced8c75e5c6a3
Successfully built fire


Training Data Extraction

In [1]:
import pandas as pd
import requests
import os
import io
from PIL import Image

# Load the CSV file
df = pd.read_csv('/kaggle/input/amazon-ml-school/student_resource 3/dataset/train.csv')


# Create a directory to store images
os.makedirs('downloaded_images', exist_ok=True)

# List to hold metadata
metadata_list = []

# Function to download images and store metadata
def download_images_and_store_metadata(image_link, group_id, entity_name, entity_value, index):
    try:
        # Download the image
        image_response = requests.get(image_link)
        image_response.raise_for_status()  # Raise an error for bad responses
        image_data = image_response.content
        
        # Save the image
        image_filename = os.path.join('downloaded_images', f"{group_id}_{index}.jpg")
        with open(image_filename, 'wb') as img_file:
            img_file.write(image_data)
        
        # Store metadata
        metadata_list.append({
            'group_id': group_id,
            'entity_name': entity_name,
            'entity_value': entity_value,
            'image_path': image_filename
        })
        print(f"Successfully downloaded: {image_link}")

    except Exception as e:
        print(f"Error processing image {image_link}: {e}")

# Process images and store metadata
for index, row in df.iterrows():
    image_link = row['image_link']
    group_id = row['group_id']
    entity_name = row['entity_name']
    entity_value = row['entity_value']
    download_images_and_store_metadata(image_link, group_id, entity_name, entity_value, index)

# Convert metadata list to DataFrame and save to CSV
metadata_df = pd.DataFrame(metadata_list)
metadata_df.to_csv('Train_metadata.csv', index=False)
print("Metadata saved to 'image_metadata.csv'.")

Successfully downloaded: https://m.media-amazon.com/images/I/71OGBt0rIML.jpg
Successfully downloaded: https://m.media-amazon.com/images/I/71w9Y6ZeeXL.jpg
Successfully downloaded: https://m.media-amazon.com/images/I/718XyYmqqJL.jpg
Successfully downloaded: https://m.media-amazon.com/images/I/81VkRJxWEIL.jpg
Successfully downloaded: https://m.media-amazon.com/images/I/814jX9pSJwL.jpg
Successfully downloaded: https://m.media-amazon.com/images/I/71pEoHrUesL.jpg
Successfully downloaded: https://m.media-amazon.com/images/I/61zSGD+X2aL.jpg
Successfully downloaded: https://m.media-amazon.com/images/I/611xamV8NLL.jpg
Successfully downloaded: https://m.media-amazon.com/images/I/61DiOFO1RBL.jpg
Successfully downloaded: https://m.media-amazon.com/images/I/61bdhyOMG0L.jpg
Successfully downloaded: https://m.media-amazon.com/images/I/71NeBifukxL.jpg
Successfully downloaded: https://m.media-amazon.com/images/I/51yyO+OGMzL.jpg
Successfully downloaded: https://m.media-amazon.com/images/I/91YTLUL7YGL.jpg

In [3]:
import pandas as pd
from paddleocr import PaddleOCR
from PIL import Image

# Initialize the OCR model (use det=True for detection and rec=True for recognition)
ocr = PaddleOCR(use_angle_cls=True, lang='en',use_gpu = True)  # Choose 'ch', 'en', 'fr', 'german', etc.

# Function to extract text from image
def extract_text(image_path):
    try:
        # Perform OCR using PaddleOCR
        result = ocr.ocr(image_path, cls=True)
        
        # Concatenate the recognized text segments
        extracted_text = ' '.join([line[1][0] for line in result[0]])

        return extracted_text.strip()
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return ""

# Apply OCR to each image and store the results in 'extracted_text'
metadata_df['extracted_text'] = metadata_df['image_path'].apply(extract_text)

# Print the updated DataFrame with extracted text
print(metadata_df)


download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 4.00M/4.00M [00:15<00:00, 253kiB/s] 


download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer/en_PP-OCRv4_rec_infer.tar


100%|██████████| 10.2M/10.2M [00:19<00:00, 532kiB/s] 


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2.19M/2.19M [00:14<00:00, 148kiB/s] 

[2024/09/15 01:14:55] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=True, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_ch

[2024/09/15 01:15:00] ppocr DEBUG: dt_boxes num : 17, elapsed : 1.3219289779663086
[2024/09/15 01:15:00] ppocr DEBUG: cls num  : 17, elapsed : 0.14403319358825684
[2024/09/15 01:15:00] ppocr DEBUG: rec_res num  : 17, elapsed : 0.2033243179321289
[2024/09/15 01:15:00] ppocr DEBUG: dt_boxes num : 14, elapsed : 0.06231236457824707
[2024/09/15 01:15:00] ppocr DEBUG: cls num  : 14, elapsed : 0.08121109008789062
[2024/09/15 01:15:00] ppocr DEBUG: rec_res num  : 14, elapsed : 0.07634258270263672
[2024/09/15 01:15:01] ppocr DEBUG: dt_boxes num : 14, elapsed : 0.059377193450927734
[2024/09/15 01:15:01] ppocr DEBUG: cls num  : 14, elapsed : 0.02110147476196289
[2024/09/15 01:15:01] ppocr DEBUG: rec_res num  : 14, elapsed : 0.0817101001739502
[2024/09/15 01:15:01] ppocr DEBUG: dt_boxes num : 38, elapsed : 0.06368541717529297
[2024/09/15 01:15:01] ppocr DEBUG: cls num  : 38, elapsed : 0.0861518383026123
[2024/09/15 01:15:01] ppocr DEBUG: rec_res num  : 38, elapsed : 0.20661354064941406
[2024/09/15

In [ ]:
metadata_df.to_csv('Trainingdataset.csv',index = False)

Testing Data Extraction

In [ ]:
import pandas as pd
import requests
import os
import io
from PIL import Image

# Load the CSV file
df = pd.read_csv('/kaggle/input/amazon-ml-school/student_resource 3/dataset/test.csv')


# Create a directory to store images
os.makedirs('downloaded_images', exist_ok=True)

# List to hold metadata
metadata_list = []

# Function to download images and store metadata
def download_images_and_store_metadata(image_link, group_id, entity_name, index):
    try:
        # Download the image
        image_response = requests.get(image_link)
        image_response.raise_for_status()  # Raise an error for bad responses
        image_data = image_response.content
        
        # Save the image
        image_filename = os.path.join('downloaded_images', f"{group_id}_{index}.jpg")
        with open(image_filename, 'wb') as img_file:
            img_file.write(image_data)
        
        # Store metadata
        metadata_list.append({
            'group_id': group_id,
            'entity_name': entity_name,
            'image_path': image_filename
        })
        print(f"Successfully downloaded: {image_link}")

    except Exception as e:
        print(f"Error processing image {image_link}: {e}")

# Process images and store metadata
for index, row in df.iterrows():
    image_link = row['image_link']
    group_id = row['group_id']
    entity_name = row['entity_name']
    download_images_and_store_metadata(image_link, group_id, entity_name, index)

# Convert metadata list to DataFrame and save to CSV
metadata_df = pd.DataFrame(metadata_list)
metadata_df.to_csv('Test_metadata.csv', index=False)
print("Metadata saved to 'image_metadata.csv'.")

In [ ]:
import pandas as pd
from paddleocr import PaddleOCR
from PIL import Image

# Initialize the OCR model (use det=True for detection and rec=True for recognition)
ocr = PaddleOCR(use_angle_cls=True, lang='en',use_gpu = True)  # Choose 'ch', 'en', 'fr', 'german', etc.

# Function to extract text from image
def extract_text(image_path):
    try:
        # Perform OCR using PaddleOCR
        result = ocr.ocr(image_path, cls=True)
        
        # Concatenate the recognized text segments
        extracted_text = ' '.join([line[1][0] for line in result[0]])

        return extracted_text.strip()
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return ""

# Apply OCR to each image and store the results in 'extracted_text'
metadata_df['extracted_text'] = metadata_df['image_path'].apply(extract_text)

# Print the updated DataFrame with extracted text
print(metadata_df)


In [ ]:
metadata_df.to_csv('Test_metadata.csv', index=False)
print("Metadata saved to 'image_metadata.csv'.")